In [ ]:
import pandas as pd
import subprocess as sb
from Bio import Entrez
import pickle
import os
from Bio import Entrez
import pickle
import traceback
import unittest

In [3]:
# We need to know for each file when the header occurs.
header_data = {"Brimr_2010.xls":(0, "Grant Number"), "Brimr_2006.xls":(4, "Project Number"), "Brimr_2011.xls":(1, "Grant Number"),
               "Brimr_2012.xls":(1, "PROJECT NUMBER"), "Brimr_2013.xls":(1, "PROJECT NUMBER"), 
               "Brimr_2007.xls":(4, "Project Number"), "Brimr_2017.xls":(1, "Project Number"), 
               "Brimr_2016.xls":(1, "PROJECT NUMBER"), "Brimr_2023.xlsx":(1, "PROJECT NUMBER"),
               "Brimr_2014.xls":(1, "Project Number"),"Brimr_2015.xls":(1, "Project Number"),
               "Brimr_2018.xls":(1, "Project Number"), "Brimr_2019.xls":(1, "PROJECT NUMBER"),
               "Brimr_2022.xls":(1, "PROJECT NUMBER"), "Brimr_2021.xls":(1, "PROJECT NUMBER"),
               "Brimr_2009.xls":(4, "Project Number"),"Brimr_2008.xls":(4, "Project Number"),
               "Brimr_2020.xls":(1, "PROJECT NUMBER") 
               
               }

In [4]:
# Let us try to load in each BRIMR file, that is load in each excel fiel in the Data Folder.

# Let us declare a dataframe that will store all of the information.
whole_df = pd.DataFrame()
for i in os.listdir("./Data"):
    print(i)
    brimr_data_year = pd.read_excel("./Data/" + i, header= header_data.get(i, (1, "Project Number"))[0])
    #print(brimr_data_year.columns)
    
    brimr_data_year = brimr_data_year[brimr_data_year[header_data.get(i,(1, "Project Number"))[1]].notna()]
    # Now that we have the year data, we really just need three different items the Organization Name, the Grant Number,
    # and the year of the data. Let us first get the first three columns.
    
    brimr_data_year.columns = [col.upper() for col in brimr_data_year.columns]
    
    if "ORGANIZATION ID (IPF)" in brimr_data_year.columns:
        del brimr_data_year["ORGANIZATION ID (IPF)"]
    if "FUNDING MECHANISM" in brimr_data_year.columns:
        brimr_data_year = brimr_data_year[brimr_data_year['FUNDING MECHANISM'] != "R&D Contracts"]

    brimr_data_year = brimr_data_year.iloc[:, :2]
    brimr_data_year.columns = ['Organization Name', 'Project Number']
    brimr_data_year['Year'] = [i.split(".")[0][-4:]] * len(brimr_data_year)
    whole_df = pd.concat([whole_df, brimr_data_year])
    # In all of the dataframes the first column is always the organization name.

whole_df = whole_df.reset_index()
    


Brimr_2011.xls
Brimr_2010.xls
Brimr_2006.xls
Brimr_2012.xls
Brimr_2013.xls
Brimr_2007.xls
Brimr_2017.xls
Brimr_2016.xls
Brimr_2023.xlsx
Brimr_2014.xls
Brimr_2015.xls
Brimr_2018.xls
Brimr_2019.xls
Brimr_2022.xls
Brimr_2021.xls
Brimr_2009.xls
Brimr_2008.xls
Brimr_2020.xls


In [ ]:
del whole_df['index']
whole_df

,Organization Name,Project Number,Year
0,"13THERAPEUTICS, INC.",R44AI065000-04A1,2011
1,"20/20 GENESYSTEMS, INC.",R44CA123994-05,2011
2,"21ST CENTURY THERAPEUTICS, INC.",261201100037C-0-0-1,2011
3,"21ST CENTURY THERAPEUTICS, INC.",R44CA141749-02,2011
4,"2B TECHNOLOGIES, INC.",R44ES016925-03,2011
...,...,...,...
1005020,"ZETROZ SYSTEMS, LLC",1R43MD015912-01,2020
1005021,"ZETROZ SYSTEMS, LLC",2R44AG061985-03,2020
1005022,"ZRT LABORATORY, LLC",3R43AT010884-01S1,2020
1005023,ZVITAMBO INSTITUTE FOR MATERNAL AND CHILD HEAL...,1R61HD103101-01,2020


In [7]:
# Let us drop rows where the Project Number is NaN.
# # Let us create a new column in the dataframe that is the correct, PUBMED 
# grant number so that we can use it in an entrez search.
def modify_pm(s:str) -> str:
    '''
    This function modifies the string from the BRIMR dataset to be in a 
    format that allows for querying of pubmed.
    '''
    try:
        s = s.split("-")[0]
        s = s[4:]
        return s
    except Exception as e:
        print(e)
        print(s)
        return None


In [8]:
# Modify the whole df so that PubMed is Queryable.
print(whole_df['Project Number'])
whole_df = whole_df[whole_df['Project Number'].notna()]
whole_df['Project Number'] = whole_df['Project Number'].apply(modify_pm)
whole_df

0             R44AI065000-04A1
1               R44CA123994-05
2          261201100037C-0-0-1
3               R44CA141749-02
4               R44ES016925-03
                  ...         
1005020        1R43MD015912-01
1005021        2R44AG061985-03
1005022      3R43AT010884-01S1
1005023        1R61HD103101-01
1005024        1R44GM136085-01
Name: Project Number, Length: 1005025, dtype: object


,Organization Name,Project Number,Year
0,"13THERAPEUTICS, INC.",I065000,2011
1,"20/20 GENESYSTEMS, INC.",A123994,2011
2,"21ST CENTURY THERAPEUTICS, INC.",01100037C,2011
3,"21ST CENTURY THERAPEUTICS, INC.",A141749,2011
4,"2B TECHNOLOGIES, INC.",S016925,2011
...,...,...,...
1005020,"ZETROZ SYSTEMS, LLC",MD015912,2020
1005021,"ZETROZ SYSTEMS, LLC",AG061985,2020
1005022,"ZRT LABORATORY, LLC",AT010884,2020
1005023,ZVITAMBO INSTITUTE FOR MATERNAL AND CHILD HEAL...,HD103101,2020


## Query By Organization
You can query by organization by replacing the variable in the cell below. The following cell performs string matching to find all organizations that contain the query word for grants. From this information we can collect a subset of information. This query will be used at a later point in the notebook to filter out results as well.

In [9]:
query = "ICAHN"

In [10]:
# Let us Create a subset of the dataframe where we look for all years for Icahn School of Medicine at Mt. Sinai
mt_sinai = whole_df[whole_df['Organization Name'].str.contains(query, case=False, na=False)]
mt_sinai

,Organization Name,Project Number,Year
335637,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,CA219321,2017
335638,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,DK046865,2017
335639,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,DK102260,2017
335640,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,OD020353,2017
335641,ICAHN SCHOOL OF MEDICINE AT MOUNT SINAI,HL131735,2017
...,...,...,...
957787,ICAHN SCHOOL OF MEDICINE MOUNT SINAI,DK113272,2020
957788,ICAHN SCHOOL OF MEDICINE MOUNT SINAI,CA249765,2020
957789,ICAHN SCHOOL OF MEDICINE MOUNT SINAI,EB026312,2020
957790,ICAHN SCHOOL OF MEDICINE MOUNT SINAI,CA232574,2020


## Year Subset
We previously used a string query to find a subset of organizations, we will perform a similar matching to filter the dataframe. To filter based on year edit the following cell variable years to be a year that you would like to subset from. You can optionally run the cells below.

In [11]:
years_filter = ['2023']

In [12]:
# Let us Create a subset of the dataframe where we look for specific years for Icahn School of Medicine at Mt. Sinai
mt_sinai = mt_sinai[mt_sinai['Year'].isin(years_filter)]
mt_sinai

,Organization Name,Project Number,Year
450734,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,DK121671,2023
450735,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,DK046865,2023
450736,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,EY029736,2023
450737,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,AR069078,2023
450738,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,AR079200,2023
...,...,...,...
451470,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,CA274967,2023
451471,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,CA269954,2023
451472,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,CA232574,2023
451473,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,DK120846,2023


## Ensuring Unique Grant IDs
We need to ensure that each grant number is unique, otherwise we will end up overcounting the number of publications sharedby two authors. 

In [ ]:
mt_sinai_uniq = mt_sinai.loc[mt_sinai.groupby("Project Number")["Year"].idxmin()]
mt_sinai

,Organization Name,Project Number,Year
450734,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,DK121671,2023
450735,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,DK046865,2023
450736,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,EY029736,2023
450737,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,AR069078,2023
450738,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,AR079200,2023
...,...,...,...
451470,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,CA274967,2023
451471,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,CA269954,2023
451472,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,CA232574,2023
451473,MOUNT SINAI ICAHN SCHOOL OF MEDICINE,DK120846,2023


## Pubmed Search
Here we utilize the pubmed api to perform a search of all the pmid's associated with each grant number. Additionally, once we have retrieved all pmids with a specific grant number, we ensure that the papers associated with those pmids are published after the grant reward date (that way we are not retrieving papers from a previously renewed grant). If we are going to search for a year that is not 2023, we would have to be cognizant of the fact that there are grant numbers that will be duplicates and we will get results from renewed grants that occur later (the farther back we go by year the more results we will have).

In [ ]:


# Set your email to comply with NCBI's usage policy
# Replace with your email address that is associated
# with your pubmed account.

# You can create an account here:
# https://account.ncbi.nlm.nih.gov/signup/?back_url=https%3A%2F%2Fpubmed.ncbi.nlm.nih.gov%2F
Entrez.email = "youremail@gmail.com"  

# Let us create two dictionaries to store associated
# pmids with grant numbers and a dictionary to associate
# each author with their co-authors and the pmids they
# share.
grant_pmids_icahn = {}
author_mapping = {}

# Iterate through the grant numbers in the dataset
for i, PM in enumerate(mt_sinai.itertuples()):  
    print(f"Processing grant {i + 1}: {PM[2]}")
    grant_number = PM[2]
    year = PM[3]
    search_query = f'"{grant_number}"[Grant Number]'
    
    try:
        # Search for PMIDs associated with the grant
        handle = Entrez.esearch(db="pubmed", term=search_query, retmax=10000)
        record = Entrez.read(handle)
        handle.close()
        
        pmids = record.get("IdList", [])
        grant_pmids_icahn[grant_number] = pmids  # Store PMIDs for the grant
        
        if pmids:
            # Fetch detailed records for the PMIDs
            handle = Entrez.efetch(db="pubmed", id=",".join(pmids), rettype="xml")
            records = Entrez.read(handle)
            handle.close()
            
            for article in records.get("PubmedArticle", []):
                pmid = article["MedlineCitation"].get("PMID", "Unknown")
                authors = article["MedlineCitation"]["Article"].get("AuthorList", [])
                if article['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']:
                    pub_year = article['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']
                if int(pub_year) < int(years_filter[0]):
                    continue
                for author in authors:
                    # Extract author details safely
                    if "Initials" in author and "LastName" in author:
                        fore_name = author["Initials"]
                        last_name = author["LastName"]
                        
                        # Handle affiliation safely
                        affiliation_info = author.get("AffiliationInfo", [])
                        if affiliation_info and isinstance(affiliation_info, list) and "Affiliation" in affiliation_info[0]:
                            affiliation = affiliation_info[0]["Affiliation"]
                        else:
                            affiliation = "Unknown"
                        
                        author_id = f"{fore_name} {last_name} ({affiliation})"
                        
                        # Initialize author entry if not present
                        if author_id not in author_mapping:
                            author_mapping[author_id] = {"coauthors": {}, "pmids": set()}
                        
                        # Add PMIDs and coauthors
                        author_mapping[author_id]["pmids"].add(pmid)
                        for coauthor in authors:
                            if coauthor != author and "Initials" in coauthor and "LastName" in coauthor:
                                co_fore_name = coauthor["Initials"]
                                co_last_name = coauthor["LastName"]
                                
                                # Handle coauthor's affiliation safely
                                co_affiliation_info = coauthor.get("AffiliationInfo", [])
                                if co_affiliation_info and isinstance(co_affiliation_info, list) and "Affiliation" in co_affiliation_info[0]:
                                    co_affiliation = co_affiliation_info[0]["Affiliation"]
                                else:
                                    co_affiliation = "Unknown"
                                
                                coauthor_id = f"{co_fore_name} {co_last_name} ({co_affiliation})"
                                author_mapping[author_id]["coauthors"][coauthor_id] = author_mapping[author_id]["coauthors"].get(coauthor_id, 0) + 1
        
        print(f"Found {len(pmids)} publications for Grant: {grant_number}, Year: {year}")
    
    except Exception as e:
        print(f"Error processing grant {grant_number}: {e}")

# Save the grant-PMID mapping and author-coauthor mapping

# -------- Change the names of these files at will. --------
with open("grant_pmids_icahn__2023.pkl", "wb") as f:
    pickle.dump(grant_pmids_icahn, f)

with open("author_mapping_icahn_2023.pkl", "wb") as f:
    pickle.dump(author_mapping, f)

print("Data processing complete. Results saved to pickle files.")


Processing grant 1: DK121671
Found 6 publications for Grant: DK121671, Year: 2023
Processing grant 2: DK046865
Found 57 publications for Grant: DK046865, Year: 2023
Processing grant 3: EY029736
Found 12 publications for Grant: EY029736, Year: 2023
Processing grant 4: AR069078
Found 21 publications for Grant: AR069078, Year: 2023
Processing grant 5: AR079200
Found 57 publications for Grant: AR079200, Year: 2023
Processing grant 6: AR069078
Found 21 publications for Grant: AR069078, Year: 2023
Processing grant 7: AR082956
Found 1 publications for Grant: AR082956, Year: 2023
Processing grant 8: HL161567
Found 5 publications for Grant: HL161567, Year: 2023
Processing grant 9: HL136255
Found 19 publications for Grant: HL136255, Year: 2023
Processing grant 10: CA205975
Found 20 publications for Grant: CA205975, Year: 2023
Processing grant 11: AR081127
Found 0 publications for Grant: AR081127, Year: 2023
Processing grant 12: DE024748
Found 8 publications for Grant: DE024748, Year: 2023
Proces

Now that we have a list of grants and authors let us process it into a schema for Neo4J. We need to create a csv file for nodes and for edges. We will only have one node type - Authors <id, label>. Id will be the author forename, lastname, and organization.  The label for each node will be the grant number associated with the Author. We will have a separate file that is for edges where each entry is of the format <source, relation,target, citations>. These two files will be authors.nodes.csv and citations.edges.csv.

## Create network via matching 'initials Lastname Affiliation'

In [ ]:
import csv

# Paths for output files
authors_nodes_file = "authors.nodes.csv"
citations_edges_file = "citations.edges.csv"

# Data for nodes
nodes_data = []
edges_data = {}


# Process author_mapping for nodes and edges
for author_id, details in author_mapping.items():
    if query in str(author_id).upper():
        # Prepare node data
        pmid_labels = ";".join(author_mapping[author_id]['pmids'])  # Join all grant numbers as a single label
        nodes_data.append({"id": author_id, "label": pmid_labels})

        # Prepare edge data
        for coauthor_id in details["coauthors"]:
            if query in str(author_id).upper() and query in str(coauthor_id).upper():
                # Create a tuple for the edge (source, target) to avoid duplication
                edge_key = tuple([author_id, coauthor_id])
                if edge_key not in edges_data:
                    edges_data[edge_key] = 0
                edges_data[edge_key] += len(details["pmids"].intersection(author_mapping[coauthor_id]["pmids"]))
    else:
        pass

# Write nodes data to CSV
with open(authors_nodes_file, mode="w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["id", "label"])
    writer.writeheader()
    writer.writerows(nodes_data)

# Write edges data to CSV
with open(citations_edges_file, mode="w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["source", "relation", "target", "Citations"])
    relation = "Coauthors"
    for (source, target), shared_citations in edges_data.items():
        writer.writerow([source, relation, target, shared_citations])

print(f"Files created: {authors_nodes_file} and {citations_edges_file}")


Files created: authors.nodes.csv and citations.edges.csv


Let us try to check to see how short the authors would be if we put all the authors with the same initials and last name into one dictionary and check the lengths of the keys.

In [27]:
nd_dic = {}
for item in nodes_data:
    item = item['ID']
    nd = item.split(" ")
    nd = nd[0:2]
    nd_dic[" ".join(nd)] = nd_dic.get(" ".join(nd), 0) + 1

print(nd_dic)

{'JJ Bieker': 2, 'TA Blenkinsop': 1, 'MC Branch': 2, 'M Weber': 2, 'MY Li': 3, 'P Flora': 4, 'E Ezhkova': 6, 'E Grossi': 3, 'CB Nguyen': 1, 'S Carcamo': 7, 'S Moran': 1, 'VK Callaú': 1, 'D Filipescu': 2, 'D Hasson': 13, 'E Bernstein': 4, 'X Zhu': 4, 'M Xu': 3, 'A Ferdinand': 1, 'JM Castellano': 6, 'SE Millar': 4, 'C Rothenberg-Lausell': 2, 'E David': 3, 'E Del': 5, 'JC Da': 2, 'D Dahabreh': 2, 'PJ Gómez-Arias': 2, 'M Nandymazumdar': 1, 'E Guttman-Yassky': 10, 'M Suárez-Fariñas': 7, 'A Obi': 1, 'S Levit': 1, 'S Jostes': 2, 'C Vardabasso': 2, 'J Dong': 2, 'R Singh': 2, 'R Phelps': 2, 'A Meadows': 3, 'M Kim': 3, 'J Bar': 2, 'G Rabinowitz': 3, 'P Facheris': 1, 'A Chang': 2, 'V Utti': 2, 'A Chowdhury': 1, 'Y Liu': 9, 'YD Estrada': 4, 'G Desman': 1, 'MB Nguyen': 2, 'M Tamari': 2, 'L Zamidar': 2, 'R Shibuya': 1, 'Z Xie': 8, 'H Iriki': 1, 'Z Wang': 7, 'K Auyeung': 2, 'G Damle': 2, 'D Demircioglu': 6, 'JK Gregory': 4, 'J Dai': 1, 'D Bogunovic': 9, 'H Hu': 1, 'BS Kim': 8, 'K Navrazhina': 1, 'S K

In [29]:
# Let us check to see how long the dictionary would be.
print(len(nd_dic.keys()))

4143


Let us now check the same for edges.

In [30]:
edge_dic = {}
for (source, target), shared_citations in edges_data.items():
    # Split source and target nodes into their components
    source_parts = source.split(" ")[0:2]  # Take the first two parts of the source
    target_parts = target.split(" ")[0:2]  # Take the first two parts of the target
    
    # Combine the processed source and target into a single key, separated by a comma
    edge_key = ", ".join([" ".join(source_parts), " ".join(target_parts)])
    
    # Count occurrences of each edge
    edge_dic[edge_key] = edge_dic.get(edge_key, 0) + 1

print(edge_dic)


{'M Weber, MC Branch': 1, 'MC Branch, MY Li': 1, 'MC Branch, P Flora': 2, 'E Ezhkova, MC Branch': 3, 'MB Nguyen, MC Branch': 1, 'M Weber, MY Li': 2, 'M Weber, P Flora': 3, 'E Ezhkova, M Weber': 3, 'M Weber, MB Nguyen': 2, 'MY Li, P Flora': 2, 'E Ezhkova, MY Li': 3, 'E Ezhkova, P Flora': 4, 'CB Nguyen, E Grossi': 1, 'E Grossi, S Carcamo': 2, 'E Grossi, S Moran': 1, 'E Grossi, VK Callaú': 1, 'D Filipescu, E Grossi': 1, 'D Hasson, E Grossi': 2, 'E Bernstein, E Grossi': 2, 'CB Nguyen, S Carcamo': 1, 'CB Nguyen, S Moran': 1, 'CB Nguyen, VK Callaú': 1, 'CB Nguyen, D Filipescu': 1, 'CB Nguyen, D Hasson': 1, 'CB Nguyen, E Bernstein': 1, 'S Carcamo, S Moran': 1, 'S Carcamo, VK Callaú': 1, 'D Filipescu, S Carcamo': 2, 'D Hasson, S Carcamo': 7, 'E Bernstein, S Carcamo': 6, 'A Agarwal, S Carcamo': 1, 'N Tung, S Carcamo': 1, 'S Carcamo, É Humblin': 1, 'MS Goldberg, S Carcamo': 1, 'NS Vyas, S Carcamo': 1, 'KG Beaumont, S Carcamo': 1, 'D Demircioglu, S Carcamo': 3, 'S Carcamo, S Sridhar': 1, 'FG Ghir

In [32]:
print(len(edge_dic))

40005


Let us create new node data and edge data where we merge by author initials and last name, since we are fairly confident that this is not the problem.

In [64]:
new_nodes_data = {}
for item in nodes_data:
    # Extract the ID and label
    item_id = item['id']
    label = item['label']
    
    # Keep only the first two parts of the ID
    prefix = " ".join(item_id.split(" ")[:2])
    
    # If the prefix already exists, merge the labels
    if prefix in new_nodes_data:
        # Join labels with a semicolon, avoiding duplicate entries
        existing_labels = set(new_nodes_data[prefix]['label'].split(";"))
        new_labels = set(label.split(";"))
        merged_labels = ";".join(sorted(existing_labels.union(new_labels)))
        new_nodes_data[prefix]['label'] = merged_labels
    else:
        # Add a new entry for this prefix
        new_nodes_data[prefix] = {"label": label}

# Convert back to a list
new_nodes_data_list = list(new_nodes_data.values())

Let us do the same for edges.

In [70]:

# Process edges to handle duplicate edges and merge PMIDs
edge_dic = {}

for (source, target), _ in edges_data.items():
    # Extract source and target and reduce to the first two parts
    source_parts = " ".join(source.split(" ")[:2])  # Take the first two parts
    target_parts = " ".join(target.split(" ")[:2])  # Take the first two parts

    # Combine the processed source and target into a unique key
    edge_key = ", ".join([" ".join(source_parts), " ".join(target_parts)])
    
    # Get PMIDs for source and target nodes
    source_pmids = set(new_nodes_data.get(source_parts, "")['label'].split(";"))
    target_pmids = set(new_nodes_data.get(target_parts, "")['label'].split(";"))
    
    # Merge PMIDs
    merged_pmids = sorted(source_pmids.union(target_pmids))
    
    # Update the edge dictionary
    if edge_key in edge_dic:
        # Combine PMIDs with existing PMIDs for this edge
        existing_pmids = set(edge_dic[edge_key]['Citations'].split(";"))
        combined_pmids = sorted(existing_pmids.union(merged_pmids))
        edge_dic[edge_key]['Citations'] = ";".join(combined_pmids)
    else:
        # Create a new entry for the edge
        edge_dic[edge_key] = {
            "source": source_parts,
            "relation": 'Coauthors',
            "target": target_parts,
            "Citations": ";".join(merged_pmids)
        }

# Convert the edge dictionary to a DataFrame
merged_edges_list = list(edge_dic.values())
merged_edges_df = pd.DataFrame(merged_edges_list)

Count number of one directional edges in edge_dic.

In [66]:
one_directional_count = 0

for edge_key in edge_dic.keys():
    # Split the key into source and target (assuming the key is formatted as "source, target")
    source, target = edge_key.split(", ")
    reverse_key = f"{target}, {source}"
    
    # Check if the reverse key exists
    if reverse_key not in edge_dic:
        one_directional_count += 1

print(f"Number of one-directional edges: {one_directional_count}")

Number of one-directional edges: 0


In [71]:
# Convert nodes data to a DataFrame
# Convert the dictionary into a DataFrame
nodes_df = pd.DataFrame.from_dict(new_nodes_data, orient='index').reset_index()
nodes_df.columns = ['id', 'label']
# Convert edges data to a DataFrame
#edges_df = pd.DataFrame(merged_edges_list)
nodes_df.to_csv('./csv2/authors.nodes.csv', index=False)
merged_edges_df.to_csv('./csv2/authors.Coauthors.authors.edges.csv', index=False)

## Fuzzy String Match by Organization

In order to fuzzy string match by organization we can utilize the rapid fuzz package.

In [34]:
from rapidfuzz import fuzz, process
# Extract and compress nodes based on the organization part
orgs = {}  # To store the unique organizations
compressed_nodes = {}

for item in nodes_data:
    # Extract the organization part
    full_id = item['ID']
    parts = full_id.split(" ")
    org_part = " ".join(parts[2:])  # Everything after the 2nd item
    
    # Find the best match among existing organizations
    best_match, score, _ = process.extractOne(org_part, orgs.keys(), scorer=fuzz.token_sort_ratio)
    
    if score >= 90:  # Threshold for similarity
        # Match found: compress to the existing organization
        compressed_org = best_match
    else:
        # No good match: treat as a new organization
        compressed_org = org_part
        orgs[compressed_org] = 1
    
    # Create compressed node mapping
    compressed_nodes[full_id] = compressed_org

print("Compressed Nodes:")
for original, compressed in compressed_nodes.items():
    print(f"{original} -> {compressed}")

TypeError: cannot unpack non-iterable NoneType object